# Conduct complex analysis with Pro mode

> #################################################################################
>
> **Note:** Pro mode is currently available only for `document` data.  
> Supported file types: pdf, tiff, jpg, jpeg, png, bmp, heif
>
> #################################################################################

This notebook demonstrates how to use **Pro mode** in Azure AI Content Understanding to enhance your analyzer with multiple inputs and optional reference data. Pro mode is designed for advanced use cases, particularly those requiring multi-step reasoning, and complex decision-making (for instance, identifying inconsistencies, drawing inferences, and making sophisticated decisions). Pro mode allows input from multiple content files and includes the option to provide reference data at analyzer creation time.

In this walkthrough, you'll learn how to:
1. Create an analyzer with a schema and reference data.
2. Analyze your files using Pro mode.

For more details on Pro mode, see the [Azure AI Content Understanding: Standard and Pro Modes](https://learn.microsoft.com/en-us/azure/ai-services/content-understanding/concepts/standard-pro-modes) documentation.

## Prerequisites
1. Ensure Azure AI service is configured following [steps](../README.md#configure-azure-ai-service-resource)
1. If using reference documents, please set up `REFERENCE_DOC_SAS_URL` and `REFERENCE_DOC_PATH` in `.env` file. You can refer to similar steps in [Set labeled data](../docs/set_env_for_labeled_data.md).
    - `REFERENCE_DOC_SAS_URL`: SAS URL for your Azure Blob container. 
    - `REFERENCE_DOC_PATH`: Folder path within the container for reference docs. 
1. Install the required packages to run the sample.

In [ ]:
%pip install -r ../requirements.txt

## Analyzer template and local files setup
- **analyzer_template**: In this sample we define an analyzer template for invoice-contract verification.
- **input_docs**: We can have multiple input document files in one folder or designate a single document file location. 
- **reference_docs**: During analyzer creation, we can provide documents that can aid in providing context that the analyzer references at inference time. We will get OCR results for these files if needed, generate a reference JSONL file, and upload these files to a designated Azure blob storage.

> For example, if you're looking to analyze invoices to ensure they're consistent with a contractual agreement, you can supply the invoice and other relevant documents (for example, a purchase order) as inputs, and supply the contract files as reference data. The service applies reasoning to validate the input documents according to your schema, which might be to identify discrepancies to flag for further review.

In [ ]:
# Define paths for analyzer template, input documents, and reference documents
analyzer_template = "../analyzer_templates/invoice_contract_verification_pro_mode.json"
input_docs = "../data/invoice_contract_verification/input_docs"
reference_docs = "../data/invoice_contract_verification/reference_docs/"

## Create Azure content understanding client
> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is utility class that contains the functions, Before the release of the Content Understanding SDK, please consider it a lightweight SDK., Fill in values for the constants **AZURE_AI_ENDPOINT**, **AZURE_AI_API_VERSION**, **AZURE_AI_API_KEY** with the information from your Azure AI Service.

> ⚠️ Important:
You must update the code below to match your Azure authentication method.
Look for the `# IMPORTANT` comments and modify those sections accordingly.
If you skip this step, the sample may not run correctly.

> ⚠️ Note: Using a subscription key works, but using a token provider with Azure Active Directory (AAD) is much safer and is highly recommended for production environments.

In [ ]:
import logging
import json
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# import utility package from python samples root directory
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

# For authentication, you can use either token-based auth or subscription key, and only one of them is required
AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
# IMPORTANT: Replace with your actual subscription key or set up in ".env" file if not using token auth
AZURE_AI_API_KEY = os.getenv("AZURE_AI_API_KEY")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2025-05-01-preview")

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    # IMPORTANT: Comment out token_provider if using subscription key
    token_provider=token_provider,
    # IMPORTANT: Uncomment this if using subscription key
    # subscription_key=AZURE_AI_API_KEY,
    # x_ms_useragent="azure-ai-content-understanding-python/pro_mode", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

## Prepare reference data
In this step, we will 
- Use Azure AI service to Extract OCR results from reference documents (if needed).
- Generate a reference `.jsonl` file.
- Upload these files to the designated Azure blob storage.

We use **REFERENCE_DOC_SAS_URL** and **REFERENCE_DOC_PATH** that's set in the Prerequisites step.

In [ ]:
# Load reference storage configuration from environment
REFERENCE_DOC_SAS_URL = os.getenv("REFERENCE_DOC_SAS_URL")
REFERENCE_DOC_PATH = os.getenv("REFERENCE_DOC_PATH")

await client.generate_knowledge_base_on_blob(reference_docs, REFERENCE_DOC_SAS_URL, REFERENCE_DOC_PATH, skip_analyze=False)

## Create analyzer with defined schema for Pro mode
Before creating the analyzer, you should fill in the constant ANALYZER_ID with a relevant name to your task. Here, we generate a unique suffix so this cell can be run multiple times to create different analyzers.

We use **REFERENCE_DOC_SAS_URL** and **REFERENCE_DOC_PATH** that's set up in the `.env` file and used in the previous step.

In [ ]:
import uuid
CUSTOM_ANALYZER_ID = "pro-mode-sample-" + str(uuid.uuid4())

response = client.begin_create_analyzer(
    CUSTOM_ANALYZER_ID,
    analyzer_template_path=analyzer_template,
    pro_mode_reference_docs_storage_container_sas_url=REFERENCE_DOC_SAS_URL,
    pro_mode_reference_docs_storage_container_path_prefix=REFERENCE_DOC_PATH,
)
result = client.poll_result(response)
if result is not None and "status" in result and result["status"] == "Succeeded":
    logging.info(f"Analyzer details for {result['result']['analyzerId']}")
    logging.info(json.dumps(result, indent=2))
else:
    logging.warning(
        "An issue was encountered when trying to create the analyzer. "
        "Please double-check your deployment and configurations for potential problems."
    )

## Use created analyzer to analyze the input documents
After the analyzer is successfully created, we can use it to analyze our input files.
> NOTE: Pro mode does multi-step reasoning and may take a longer time to analyze.

In [ ]:
response = client.begin_analyze(CUSTOM_ANALYZER_ID, file_location=input_docs)
result_json = client.poll_result(response, timeout_seconds=360)

logging.info(json.dumps(result_json, indent=2))

## Delete exist analyzer in Content Understanding Service
This snippet is not required, but it's only used to prevent the testing analyzer from residing in your service. Without deletion, the analyzer will remain in your service for subsequent reuse.

In [ ]:
client.delete_analyzer(CUSTOM_ANALYZER_ID)